In [1]:
import os

In [2]:
%pwd

'c:\\Projects\\Text-Summarization-using-NLP\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Projects\\Text-Summarization-using-NLP'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [6]:
from textSummarization.constants import *
from textSummarization.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name

        )

        return model_evaluation_config

In [10]:
import transformers
print(transformers.__version__)

4.46.3


In [11]:
pip uninstall transformers -y

Found existing installation: transformers 4.46.3Note: you may need to restart the kernel to use updated packages.

Uninstalling transformers-4.46.3:
  Successfully uninstalled transformers-4.46.3


In [12]:
pip install transformers


  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.46.3-py3-none-any.whl (10.0 MB)


In [11]:
pip uninstall datasets -y

Found existing installation: datasets 3.1.0
Uninstalling datasets-3.1.0:
  Successfully uninstalled datasets-3.1.0
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install datasets

  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
Using cached datasets-3.1.0-py3-none-any.whl (480 kB)
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install evaluate

  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
Using cached evaluate-0.4.3-py3-none-any.whl (84 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install rouge_score

Note: you may need to restart the kernel to use updated packages.


In [13]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
from rouge_score import rouge_scorer
import torch
import pandas as pd
from tqdm import tqdm

In [17]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """Split the dataset into smaller batches that we can process simultaneously.
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def calculate_metric_on_test_ds(self, dataset, scorer, model, tokenizer, 
                                   batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                                   column_text="article", column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        all_scores = { "rouge1": [], "rouge2": [], "rougeL": [] }  # Removed 'rougeLsum' because it's not available

        for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024, truncation=True, 
                               padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                       attention_mask=inputs["attention_mask"].to(device), 
                                       length_penalty=0.8, num_beams=8, max_length=128)
            
            # Decode the generated texts
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                                  clean_up_tokenization_spaces=True) 
                                 for s in summaries]
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            # Calculate ROUGE scores for each summary
            for pred, ref in zip(decoded_summaries, target_batch):
                scores = scorer.score(ref, pred)
                for key in all_scores:
                    all_scores[key].append(scores[key].fmeasure)

        # Calculate average ROUGE scores
        avg_scores = {key: sum(values) / len(values) for key, values in all_scores.items()}
        return avg_scores

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
        # Loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # Initialize RougeScorer
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

        # Evaluate the model and calculate ROUGE scores
        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'][0:10], scorer, model_pegasus, tokenizer, 
            batch_size=2, column_text='dialogue', column_summary='summary'
        )

        # Save scores to CSV
        df = pd.DataFrame(score, index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)


In [18]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2025-02-17 19:30:31,963: INFO: common: yaml file: config\config.yaml loaded successfully]


[2025-02-17 19:30:32,127: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-17 19:30:32,151: INFO: common: created directory at: artifacts]
[2025-02-17 19:30:32,159: INFO: common: created directory at: artifacts/model_evaluation]
[2025-02-17 19:30:46,890: INFO: rouge_scorer: Using default tokenizer.]


100%|██████████| 5/5 [16:44<00:00, 200.86s/it]
